In [ ]:
import ast
import json
import os

from dotenv import load_dotenv
from openai import OpenAI

In [ ]:
load_dotenv(".env")
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key)

In [ ]:
def get_completion(prompt, model="gpt-4o-mini"):
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        max_tokens=100,
        temperature=0.9,
        response_format={
            "type": "json_object",
            # "type": "text",
        },
    )

    completion = completion.to_dict()

    content = completion["choices"][0]["message"]["content"]

    return content

#### Average prompt, average response

**Task**: run the following prompt and examine the response

In [ ]:
prompt = """
Can I have a list of product names for a pair of shoes that can fit any foot size?
"""

get_completion(prompt)

**Task**: what do you think some of the obvious issues with this prompt?

1. **Vague direction**: what style and attributes?
2. **Unformatted output**: how many names and in what format?
3. **Missing example**: what good names look like?
4. **Limited evaluation**: how to determine if a name is good or bad?
5. **No task division**: what are the steps involved?

### The Five Principles of Prompting

1. **Give Direction**: describe the desired style in detail, or reference a relevant persona

2. **Specify Format**: define what rules to follow, and the required structure of the response

3. **Provide Examples**: insert a diverse set of test cases where the task was done correctly

4. **Evaluate Quality**: identify errors and rate responses, testing what drives performance

5. **Divide Labour**: split tasks into multiple steps, chained together for complex goal

#### 1. **Give Direction**

Human would also struggle to complete this task without a good brief. Imagine what context a human might need for this task and try including it in the prompt.

**Strategies**:

* Role-playing
* Prewarming or internal retrieval
* Best advice

**Role-playing**: find a persona (who is famous in the training data) to emulate their style

**Task**: Use Steve Jobs as a persona to give direction

In [ ]:
prompt = "Brainstorm a list of product names for a shoe that fits any foot size, in the style of Steve Jobs?"
get_completion(prompt)

**Prewarming or internal retrieval**: ask chatGPT for best practice advice, then ask it to follow its own advice 

**Task**: ask chatGPT for advice, then use its advice

In [ ]:
prompt = "Please give me 5 tips for naming products based on expert industry advice"
advice = get_completion(prompt)
advice

In [ ]:
prompt = f"""
Using the following advice:

{advice}

Brainstorm a list of product names for a shoe that fits any foot size?
"""

get_completion(prompt)

**Best advice**: take the best advice you can find for the task and use it

**Task**: Use the [5 Golden Rules](https://www.brandwatch.com/blog/how-to-name-a-product-our-5-golden-rules/) for naming a product

In [ ]:
advice = """
1. It should be readable and writable
If your product name is hard to pronounce, people won’t talk about it and if they can’t write it down (and spell it correctly!) when they hear it, how do you expect them to Google it?

Keep it simple and don’t go with any wacky spellings just for the sake of it.

2. It should be unique
It’s very hard in this day and age to be completely unique, so you can give yourself a bit of leeway, but your product name should at least be unique to your industry.

This makes it much easier to get the domain, do well in search and know that when someone says the name, they mean your product.

3. It should be short, punchy and memorable
The longer the name, the harder it is to grab people.

Longer names also mean people resort to abbreviations that you often don’t get to control.

4. It should look good written down and sound cool to say
You want your product name to jump off the page and stand out next to all the other boring words around it.

When someone says it in a sentence it should stand out so everyone around pays attention.

5. It should evoke an emotion, feeling or idea
Your product name should tie back into what your product is, what the feeling you want people to have when experiencing your product is, and/or what idea are you trying to get across.

It should be emotive and inspiring
"""

In [ ]:
prompt = f"""
Using the following 5 golden rules:

{advice}

Brainstorm a list of product names for a shoe that fits any foot size?
"""

get_completion(prompt)

**Trade-off**: too much direction can cause the model to quickly get into a conflicting combination that it can't resolve. While too much direction can narrow the creativity of the model, too little direction is the more common problem

#### 2. **Specify Format**


AI models are universal translators! They can translate between:

* Languages, for example French to English
* Data formats, for example lists to json 
* Programming languages, for example, for example C# to Python
* Natural language and programming languages, for example text to Python

* **Task**: Update the prompt to return the response as a comma-separated list of 5 names

In [ ]:
prompt = """
Can I have a list of product names for a pair of shoes that can fit any foot size?

Return the response as a comma-separated list of 5 names, in this format: ["Name 1", "Name 2", "Name 3", "Name 4", "Name 5"]
"""

completion = get_completion(prompt)
completion

* **Task**: Print the items in the list, one by one
* *Hint*: the response is a string list, use `ast.literal_eval` first to convert it into a list object

In [ ]:
completion = ast.literal_eval(completion)

for item in completion:
    print(item)

* **Task**: Update the prompt to return the response as a `json` where the `key` is `"ProductNames"` and the value is a comma-separated list of 5 names

In [ ]:
prompt = """
Can I have a list of product names for a pair of shoes that can fit any foot size?

Return the response as a json in this format: 
{"ProductNames": ["Name 1", "Name 2", "Name 3", "Name 4", "Name 5"]}

Return only json

"""

completion = get_completion(prompt)
completion

In [ ]:
completion = ast.literal_eval(completion)

completion

* **Task**: Use the `response_format={"type": "json_object"}` parameters in the completion request to natively return a `json` response

In [ ]:
prompt = """
Can I have a list of product names for a pair of shoes that can fit any foot size?

Return the response as a json in this format: 
{"ProductNames": ["Name 1", "Name 2", "Name 3", "Name 4", "Name 5"]}
"""

completion = get_completion(prompt)
completion

* **Task**: Print the items in the `ProductNames`
* *Hint*: The response is a string json, use `json.loads` to convert it into a json object first

In [ ]:
completion = json.loads(completion)

completion["ProductNames"]

#### 3. **Provide Examples**

* Our prompts didn't give any examples of what "good" outputs look like! A prompt with no examples is called a `zero-shot` prompt which is asking for a lot without giving much in return.

* Even adding one example, which is called `one-shot`, helps considerably

* It's more common practice to add multiple examples, which is called `few-shot`. The strength of a prompt often comes down to the examples used, especially if you're not a domain expert as providing examples can sometimes be easier than trying to explain what is it that you like about the output to look like 

* **Task**: experiment with the number of examples, from `zero-shot` to `few-shot`

In [ ]:
prompt = """

"""

completion = get_completion(prompt)
completion = json.loads(completion)

completion["ProductNames"]

**Trade-off**: between reliability and creativity. Go past 3-5 examples and the results will become more reliable but less creative. Give similar examples and the results will be much more constrained and less diverse. Lack of diversity and variations can cause a problem in handling edge cases.